# Implement Spatial Pooler

В данной тетради требуется реализовать Spatial Pooler по аналогии с [описанием](https://numenta.com/assets/pdf/spatial-pooling-algorithm/Spatial-Pooling-Algorithm-Details.pdf) и [статьёй](https://www.frontiersin.org/articles/10.3389/fncom.2017.00111/pdf) от нументы. Данная тетрадь сделана на основе [примера](https://github.com/htm-community/htm.core/blob/master/py/htm/examples/mnist.py) из нументовского фреймворка `htm.core`.

Для начала посмотри эпизоды 0-8 видео гайда [HTMSchool](https://www.youtube.com/watch?v=XMB0ri4qgwc&list=PL3yXMgtrZmDqhsFQzwUC9V8MeeVOQ7eZ9).

## 01. Getting ready

Данная секция содержит:

- [опционально] установка `htm.core`
- импорт необходимых пакетов (убедись, что все они установлены)
- загрузка датасета

### HTM.Core

Если у тебя не установлен пакет `htm.core`, раскомментируй и запусти следующую ячейку. В случае проблем, обратись к официальной странице пакета на [гитхабе](https://github.com/htm-community/htm.core#installation) и проверь требуемые зависимости.

In [ ]:
# !python -m pip install -i https://test.pypi.org/simple/ htm.core

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix

from htm.bindings.algorithms import SpatialPooler
from htm.bindings.sdr import SDR, Metrics

%matplotlib inline
    
seed = 1337

### Load data

Следующая ячейка загружает датасет MNIST (займет порядка 10-20 сек).

In [ ]:
def load_ds(name, num_test, shape=None):
    """ 
    fetch dataset from openML.org and split to train/test
    @param name - ID on openML (eg. 'mnist_784')
    @param num_test - num. samples to take as test
    @param shape - new reshape of a single data point (ie data['data'][0]) as a list. Eg. [28,28] for MNIST
    """
    data = fetch_openml(name, version=1)
    sz=data['target'].shape[0]

    X = data['data']
    if shape is not None:
        new_shape = shape.insert(0, sz)
        X = np.reshape(X, shape)

    y = data['target'].astype(np.int32)
    # split to train/test data
    train_labels = y[:sz-num_test]
    train_images = X[:sz-num_test]
    test_labels  = y[sz-num_test:]
    test_images  = X[sz-num_test:]

    return train_labels, train_images, test_labels, test_images


def shuffle_data(x, y):
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x, y = np.array(x), np.array(y)
    return x[indices], y[indices]


train_labels, train_images, test_labels, test_images = load_ds('mnist_784', 10000, shape=[28,28])

np.random.seed(seed)
train_images, train_labels = shuffle_data(train_images, train_labels)
test_images, test_labels = shuffle_data(test_images, test_labels)

n_train_samples = train_images.shape[0]
n_test_samples = test_images.shape[0]
image_shape = train_images[0].shape
image_side = image_shape[0]
image_size = image_side ** 2


train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

Пример формата данных датасета

In [ ]:
plt.imshow(train_images[0])
print(f'Label: {train_labels[0]}')
print(f'Image shape: {image_shape}')
print(f'Image middle row: {train_images[0][image_side//2]}')

Перекодируем датасет в бинарные изображения и дальше будем работать с бинарными данными.

In [ ]:
def plot_flatten_image(flatten_image, image_height=28):
    plt.imshow(flatten_image.reshape((image_height, -1)))

def to_binary_flatten_images(images):
    n_samples = images.shape[0]
    # flatten every image to vector
    images = images.reshape((n_samples, -1))
    # binary encoding: each image pixel is encoded either 0 or 1 depending on that image mean value
    images = (images >= images.mean(axis=1, keepdims=True)).astype(np.int8)
    return images


train_images = to_binary_flatten_images(train_images)
test_images = to_binary_flatten_images(test_images)
plot_flatten_image(train_images[0])

## 02. Baseline: classifier on raw input

В качестве бейзлайна возьмем стандартный sklearn'овский логрег классификатор

In [ ]:
%%time

def test_bare_classification(x_tr,  y_tr, x_tst, y_tst):
    linreg = LogisticRegression(tol=.001, max_iter=100, multi_class='multinomial', penalty='l2', solver='lbfgs', n_jobs=3)
    linreg.fit(x_tr, y_tr)
    
    score = linreg.predict(x_tst) == y_tst
    score = score.mean()
    print('Score:', 100 * score, '%')
    return score


np.random.seed(seed)
# for debug purposes I chose smaller subset of the train/test set, you can set the whole set of 60k training samples
n = 1000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

# 1k: 87.3; 888ms
# 60k: 92.11; 38s
test_bare_classification(x_tr, y_tr, x_tst, y_tst)

## 03. Spatial Pooler: skeleton

Временно сделаем пустой класс Spatial Pooler'а заглушку, чтобы дальше ввести весь необходимый auxiliary код для обучения и тестирования.

In [ ]:
class NoOpSpatialPooler:
    def __init__(self, input_size):
        self.input_size = input_size
        self.output_size = input_size
        
    def compute(self, dense_sdr, learn):
        return np.nonzero(dense_sdr)[0]
        

np.random.seed(seed)
sp = NoOpSpatialPooler(train_images[0].size)
sparse_sdr = sp.compute(train_images[0], True)

print(sparse_sdr.size, sp.output_size)
assert sparse_sdr.size < sp.output_size

## 04. Train/test SP performance aux pipeline

Ниже непосредственно код для обучения и тестирования классификации с использованием Spatial Pooler'а. Общая схема следующая - мы обучаем SP на train set'е, а дальше SDR векторы на выходе из SP используем в кач-ве входных данных для логрег классификатора в надежде, что эти данные разделимы еще лучше.

__NB__: Не удивляйся, в реализации ниже обучение идет в немного полуонлайн режиме - делается небольшой претрейн, а потом полностью онлайн. Почему? прост))

In [ ]:
%%time

def pretrain_sp(sp, images, n_samples):
    for img in images[:n_samples]:
        sp.compute(img, True)
    
def encode_to_csr_with_sp(images, sp, learn):
    flatten_encoded_sdrs = []
    indptr = [0]
    for img in images:
        encoded_sparse_sdr = sp.compute(img, learn)
        flatten_encoded_sdrs.extend(encoded_sparse_sdr)
        indptr.append(len(flatten_encoded_sdrs))

    data = np.ones(len(flatten_encoded_sdrs))
    csr = csr_matrix((data, flatten_encoded_sdrs, indptr), shape=(images.shape[0], sp.output_size))
    return csr

def test_classification_with_sp(x_tr,  y_tr, x_tst, y_tst, sp):
    # a small pretrain SP before real work
    pretrain_sp(sp, x_tr, n_samples=1000)
    
    # encode images and continuously train SP
    csr = encode_to_csr_with_sp(x_tr, sp, learn=True)
    
    # train linreg
    linreg = LogisticRegression(tol=.001, max_iter=100, multi_class='multinomial', penalty='l2', solver='lbfgs', n_jobs=3)
    linreg.fit(csr, y_tr)
    
    # encode test images (without SP learning) and then test score
    csr = encode_to_csr_with_sp(x_tst, sp, False)
    score = linreg.predict(csr) == y_tst
    score = score.mean()
    print('Score:', 100 * score, '% for n =', len(x_tr))
    return score

n = 1000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]
my_sp = NoOpSpatialPooler(train_images[0].size)

# 87.3; 1.16s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, my_sp)

## 05. Spatial Pooler: learning

__Здесь начинается практическое задание__

В этой части требуется реализовать простую версию SP с обучением как описано в видео htm scool до бустинга (т.е. бустинг пока не нужен).

In [ ]:
class LearnableSpatialPooler:
    def __init__(
        self, input_size, output_size, permanence_threshold, output_sparsity, synapse_permanence_deltas, receptive_field_sparsity
    ):
        '''
        params:
            `input_size` - the size of the input SDR
            `output_size` - the size of the output SDR
            `permanence_threshold` - value in [0, 1], defines whether or not a connection is active
            `output_sparsity` - value in [0, 1], defines desired output SDR sparsity, e.g. 0.02 is 2% sparsity
            `synapse_permanence_deltas` - tuple of (p+, p-), defines permanence increment/decrement for learning
            `receptive_field_sparsity` - value in [0, 1], defines the fraction of _potential_ synapses
        '''
        self.input_size = input_size
        self.output_size = output_size
        
        # TODO: initialization        
        
        
    def compute(self, dense_sdr, learn):
        '''
        params:
            `dense_sdr` - input SDR in dense for, i.e. as np.array
            `learn` - bool flag, whether or not to do a learning step
        
        returns:
            a list of activated columns indices
        '''
        
        # TODO: compute and learn

        # make sure that activated_cols is array of indices, not a whole binary vector
        return activated_cols

        
np.random.seed(seed)
sp = LearnableSpatialPooler(
    input_size=train_images[0].size,
    output_size=10**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8
)
sparse_sdr = sp.compute(train_images[0], True)

print(sparse_sdr.size, sp.output_size, sp.n_active_bits)
assert sparse_sdr.size == sp.n_active_bits
sparse_sdr

### 05.1. Naive SP performance

Проверь качество работы своей реализации. В комментариях примерные значения для разных `n`, на которые можно попробовать ориентироваться:

In [ ]:
%%time

np.random.seed(seed)
n = 1000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

sp = LearnableSpatialPooler(
    input_size=train_images[0].size, 
    output_size=30**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8
)
# 1k: 80.2; 2.98s
# 60k: 89.3; 72 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)

Интересная метрика оценки качества работы SP и вырожденности датасета - энтропия выходных активаций.

Добавь либо в реализацию класса, либо где-то сбоку возможность учета статистики активаций выходных клеток (ты наверняка заметил, что обычно их контринтуитивно называют столбцами, что оч круто запутывает) и функцию подсчета энтропии на основе этой статистики. Собери такую статистику на одном прогоне на всем датасете и:

- нарисуй гистограмму вероятностей активаций
- посчитай энтропию $H = -\sum p \cdot \log p$

In [ ]:
# TODO: prob hist & entropy

Теперь давай посмотрим, как выходной размер `output_size` влияет на качество работы. Протестируй разные значения `output_size` - например, $[10^2, 15^2,..., 55^2, 60^2]$ и отрисуй график зависимости [качества классификации, но можешь и времени работы тоже].

_Можно для начала прикинуть для n=1000 и двигаться дальше, а позже пересчитать на всем датасете_

In [ ]:
# TODO: test and plot results

## 06. Spatial Pooler: boosting

В этом пункте требуется реализовать бустинг. Обрати внимание, в видео htm school речь ведется о бустинге overlap score на основе активности выходных ячеек -  именно такой вид бустинга и нужно реализовать. Еще есть бустинг значений permanence синапсов выходных ячеек, имеющих слишком низкое среднее значение overlap score за последние N итераций - рассмотрение этого вида бустинга оставим в стороне. Так что дальше под бустингом будет иметься в виду только первый вариант.

In [ ]:
class BoostedSpatialPooler:
    def __init__(
        self, input_size, output_size, permanence_threshold, output_sparsity, synapse_permanence_deltas, receptive_field_sparsity,
        max_boost_factor, boost_sliding_window
    ):
        '''
        params:
            `input_size` - the size of the input SDR
            `output_size` - the size of the output SDR
            `permanence_threshold` - value in [0, 1], defines whether or not a connection is active
            `output_sparsity` - value in [0, 1], defines desired output SDR sparsity, e.g. 0.02 is 2% sparsity
            `synapse_permanence_deltas` - tuple of (p+, p-), defines permanence increment/decrement for learning
            `receptive_field_sparsity` - value in [0, 1], defines the fraction of _potential_ synapses
            `max_boost_factor` - value in [1, +inf), defines maximum allowed boosting
            `boost_sliding_window` - value in [1, +inf), defines the size of the window for moving avg output column activity
        '''
        self.input_size = input_size
        self.output_size = output_size
        
        # TODO: initialization
        
    def compute(self, dense_sdr, learn):
        '''
        params:
            `dense_sdr` - input SDR in dense for, i.e. as np.array
            `learn` - bool flag, whether or not to do a learning step
        
        returns:
            a list of activated columns indices
        '''
        
        # TODO: compute and learn

        # make sure that activated_cols is array of indices, not a whole binary vector
        return activated_cols
        

np.random.seed(seed)
my_sp = BoostedSpatialPooler(
    train_images[0].size, 10**2, .5, .04, (.1, .02), receptive_field_sparsity=.8, 
    max_boost_factor=1.5, boost_sliding_window=1000
)
my_sp.compute(train_images[0], True)

### 06.1. SP with boosting performance

_Дальше все как и в пункте 05.1_

Проверь качество работы своей реализации. В комментариях по-прежнему примерные значения для разных `n`, на которые можно попробовать ориентироваться:

In [ ]:
%%time

np.random.seed(seed)
n = 1000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

sp = BoostedSpatialPooler(
    input_size=train_images[0].size, 
    output_size=30**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8,
    max_boost_factor=3,
    boost_sliding_window=1000
)
# 1k: 84.0; 3.24 s
# 60k: 91.15; 86 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)

- нарисуй гистограмму вероятностей активаций
- посчитай энтропию $H = -\sum p \cdot \log p$
- сравни с результатами без бустинга - есть какие-то очевидные выводы?

In [ ]:
# TODO: prob hist & entropy & comparison

Протестируй разные значения `output_size` - например, $[10^2, 15^2,..., 55^2, 60^2]$ и отрисуй график зависимости [качества классификации, но можешь и времени работы тоже].

Сравни с результатами без бустинга

In [ ]:
# TODO: test and plot results & comparison

___
__[Опционально]__ можешь ради интереса поиграться с таким параметром как `receptive_field_sparsity`. Возьми в качестве выходного размера `output_size` = $50^2$ и проверь результаты на разных значениях параметра $[0.4, 0.5, ..., 1.0]$ и построй график зависимости.

In [ ]:
# TODO: test receptive_field_sparsity

## 07. TESTING

Дальше просто сравнительное тестирование получившихся результатов.

In [ ]:
%%time

def test_bare_classification(x_tr,  y_tr, x_tst, y_tst):
    linreg = LogisticRegression(tol=.001, max_iter=100, multi_class='multinomial', penalty='l2', solver='lbfgs', n_jobs=3)
    linreg.fit(x_tr, y_tr)
    
    score = linreg.predict(x_tst) == y_tst
    score = score.mean()
    print('Score:', 100 * score, '%')
    return score

np.random.seed(seed)
n = 100000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

# 92.11; 38s
test_bare_classification(x_tr, y_tr, x_tst, y_tst)

In [ ]:
%%time

np.random.seed(seed)
n = 100000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

sp = LearnableSpatialPooler(
    input_size=train_images[0].size, 
    output_size=30**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8,
)
# 89.3; 72 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)

In [ ]:
%%time

np.random.seed(seed)
n = 100000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

sp = LearnableSpatialPooler(
    input_size=train_images[0].size, 
    output_size=50**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8,
)
# 93.15; 221 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)

In [ ]:
%%time

np.random.seed(seed)
n = 100000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]

sp = BoostedSpatialPooler(
    input_size=train_images[0].size, 
    output_size=30**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8,
    max_boost_factor=3,
    boost_sliding_window=1000
)
# 91.15; 86 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)

In [ ]:
%%time

np.random.seed(seed)
n = 100000
x_tr, y_tr = train_images[:n], train_labels[:n]
x_tst, y_tst = test_images[:n], test_labels[:n]


sp = BoostedSpatialPooler(
    input_size=train_images[0].size, 
    output_size=50**2,
    permanence_threshold=.5,
    output_sparsity=.04,
    synapse_permanence_deltas=(.1, .03),
    receptive_field_sparsity=.8,
    max_boost_factor=3,
    boost_sliding_window=1000
)
# 94.44; 238 s
test_classification_with_sp(x_tr, y_tr, x_tst, y_tst, sp)